<a href="https://colab.research.google.com/github/ShadaFathima/ML-DL/blob/main/TextSummarization_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets tensorflow sentencepiece evaluate



In [ ]:
import tensorflow as tf
from datasets import load_dataset
from transformers import T5Tokenizer, TFT5ForConditionalGeneration, create_optimizer
import evaluate
import numpy as np

In [ ]:
# Load dataset (streaming for low memory usage)
stream_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train", streaming=True)
val_stream = load_dataset("cnn_dailymail", "3.0.0", split="validation", streaming=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Load tokenizer & model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("t5-small", use_safetensors=False)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
# Hyperparameters
max_input_length = 512
max_target_length = 150
batch_size = 4
learning_rate = 2e-4
num_train_steps = 5000  # increase for better performance

In [ ]:
# -----------------------------
# Preprocessing Function
# -----------------------------
def preprocess_streamed(example):
    inputs = "summarize: " + example["article"]
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding="max_length",
        return_tensors="tf"
    )

    labels = tokenizer(
        example["highlights"],
        max_length=max_target_length,
        truncation=True,
        padding="max_length",
        return_tensors="tf"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:

# -----------------------------
# Batch Generator
# -----------------------------
def batch_generator(stream_dataset, batch_size=batch_size):
    batch = []
    for example in stream_dataset:
        processed = preprocess_streamed(example)
        batch.append(processed)
        if len(batch) == batch_size:
            yield {
                "input_ids": tf.concat([b["input_ids"] for b in batch], axis=0),
                "attention_mask": tf.concat([b["attention_mask"] for b in batch], axis=0),
                "labels": tf.concat([b["labels"] for b in batch], axis=0)
            }
            batch = []
    if batch:
        yield {
            "input_ids": tf.concat([b["input_ids"] for b in batch], axis=0),
            "attention_mask": tf.concat([b["attention_mask"] for b in batch], axis=0),
            "labels": tf.concat([b["labels"] for b in batch], axis=0)
        }

train_dataset = tf.data.Dataset.from_generator(
    lambda: batch_generator(stream_dataset),
    output_signature={
        "input_ids": tf.TensorSpec(shape=(None, max_input_length), dtype=tf.int32),
        "attention_mask": tf.TensorSpec(shape=(None, max_input_length), dtype=tf.int32),
        "labels": tf.TensorSpec(shape=(None, max_target_length), dtype=tf.int32),
    }
).take(5000).prefetch(tf.data.AUTOTUNE)  # process ~2000 samples for a mini fine-tune


In [ ]:
# -----------------------------
# Optimizer & Compile
# -----------------------------
optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_warmup_steps=500,
    num_train_steps=num_train_steps,
)
model.compile(optimizer=optimizer)


In [ ]:
# -----------------------------
# Train
# -----------------------------
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("t5_cnn_best", save_best_only=True, monitor="loss"),
    tf.keras.callbacks.EarlyStopping(monitor="loss", patience=2, restore_best_weights=True),
]

model.fit(train_dataset, epochs=10, steps_per_epoch=500, callbacks=callbacks)

model.save_pretrained("t5summarizerfinal")
tokenizer.save_pretrained("t5summarizerfinal")


Epoch 1/10


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


500/500 [==============================] - 239s 342ms/step - loss: 2.3622
Epoch 2/10
500/500 [==============================] - 164s 328ms/step - loss: 0.9614
Epoch 3/10
500/500 [==============================] - 170s 341ms/step - loss: 0.9522
Epoch 4/10
500/500 [==============================] - 155s 310ms/step - loss: 0.9386
Epoch 5/10
500/500 [==============================] - 120s 239ms/step - loss: 0.9607
Epoch 6/10
500/500 [==============================] - 120s 239ms/step - loss: 0.9526


('t5summarizerfinal/tokenizer_config.json',
 't5summarizerfinal/special_tokens_map.json',
 't5summarizerfinal/spiece.model',
 't5summarizerfinal/added_tokens.json')

In [ ]:
!pip install rouge_score

In [ ]:
# -----------------------------
# Evaluation using ROUGE
# -----------------------------
rouge = evaluate.load("rouge")

def generate_summary(example):
    inputs = tokenizer(
        "summarize: " + example["article"],
        return_tensors="tf",
        truncation=True,
        padding="max_length",
        max_length=max_input_length
    )
    summary_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_target_length,
        num_beams=4,
        length_penalty=2.0,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

predictions, references = [], []
for i, example in enumerate(val_stream):
    pred = generate_summary(example)
    predictions.append(pred)
    references.append(example["highlights"])
    if i >= 49:  # evaluate on 50 samples
        break

rouge_score = rouge.compute(predictions=predictions, references=references)
print("ROUGE Score:", rouge_score)

ROUGE Score: {'rouge1': np.float64(0.3110377055218291), 'rouge2': np.float64(0.1343167742357163), 'rougeL': np.float64(0.23108710978498326), 'rougeLsum': np.float64(0.2670348822362975)}
